In [128]:
# This tutorial does not capture all functionality
# Rather, this tutorial is a quickstart guide for some
#    features of the model

In [129]:
# Import modules
from Engine_SimObjects import MultiTierPolicy
from Engine_DataObjects import DataPrepConfig, TierInfo
from Engine_SimModel import SimModelConstructor

import numpy as np
import json

from pathlib import Path

# Import time to analyze simulation execution time
import time

In [130]:
DataPrepConfig.base_path = Path()

In [131]:
# Let's demonstrate how to run the Austin model

# Our data files live in a folder called "austin_data"
# So we update our base path 
DataPrepConfig.base_path = DataPrepConfig.base_path / "austin_data"

# To manage all the filenames, we have a .json file called "filenames.json"
austin_dict_filenames = json.load(open(DataPrepConfig.base_path / "filenames.json"))

In [132]:
# Let's construct our model!

# We create an instance of SimModelConstructor, which combines all the 
#    data in the filenames given by austin_dict_filenames nicely for us
# simulation_start_date_str must be in the format "MM/DD/YY"
# max_simulation_length (in simulation days) is the number of days
#    for which we have calendar data
sim_model_constructor = SimModelConstructor(city_name="austin",
                                            simulation_start_date_str="02/28/20",
                                            max_simulation_length=963,
                                            dict_filenames=austin_dict_filenames)

# Note that before we create the Austin model, we can toggle different attributes
#    of sim_model_constructor to modify the Austin model to our liking
# For example, we can manually override any attached data / filenames
#    (not shown here)

# Here's our austin model, which is an instance of the class SimModel
austin_model = sim_model_constructor.create_sim_model()

In [133]:
# Our SimModel class by default creates 4 instances of VaccineGroup
# The vaccine groups correspond to unvaccinated, first dose,
#    second dose, and waned immunity respectively.
# Each VaccineGroup instance has compartments
#    S, E, IA, IY, PA, PY, R, D, IH, ICU, and each
#    such compartment is a 5x2 matrix (5 age groups,
#    2 risk groups)

# Check it out -- the first group is named "unvax"
austin_model.vaccine_groups[0].v_name

'unvax'

In [134]:
# At any time, we can also access the "current" number in each compartment

# Attributes in our SimModel instance are updated in-place
#   to reflect the most current simulation state.

# Let's see the number susceptible in the unvaccinated group
#    (we have not run the simulation yet, so these are initialized
#    values)
austin_model.vaccine_groups[0].S

array([[128527.,   9350.],
       [327148.,  37451.],
       [915893., 156209.],
       [249273., 108196.],
       [132505., 103763.]])

In [135]:
# Let's attach a staged-alert policy to our model (4 stages)
# The way to interpret this is that when there are more than 100 daily 7-day-avg hospital admissions,
#    we move to stage 2. When there are more than 200 daily 7-day-avg admits, we move to stage 3, etc...
thresholds = (-1, 100, 200, 500)

# The corresponding transmission reductions for each stage are given in a json file
# This data is stored in a TierInfo object
tiers = TierInfo(austin_dict_filenames["tier_info_json"])

# Now we can create a staged-alert policy
staged_alert_policy = MultiTierPolicy(tiers, thresholds)

# Let's attach it to our simulation model
austin_model.policy = staged_alert_policy

# Let's simulate our model for 200 days with this policy and
#    see how this same compartment has changed
austin_model.simulate_time_period(200)

# Note that all values of S for the unvaccinated are lower than before
austin_model.vaccine_groups[0].S

array([[126030.,   9148.],
       [317130.,  36327.],
       [886673., 151180.],
       [242722., 105299.],
       [130756., 102371.]])

In [136]:
# Note we can continue simulating after we left off
# This allows us to simulate in "chunks" and check our work
#    or obtain statistics in between simulation chunks

# The following is 200 (since we simulated until day 200 previously)
austin_model.next_t

# Now we start where we left off, and simulate from 200 to 945
austin_model.simulate_time_period(945)

In [137]:
# We can also query whether the specified policy is
#   feasible, i.e. whether it prevents an ICU capacity violation.
#   Note that we check for an ICU capacity violation from
#   timepoints fixed_kappa_end_date onwards. See below for
#   more explanation of fixed_kappa_end_date.
print(austin_model.compute_feasibility())

False


In [138]:
# To run a new replication on the same model,
#    we can use the same SimModel instance. We just have to 
#    reset any stored simulation history.
austin_model.reset()

# If we want to test the same policy on a different sample path,
#   we can still use the same policy object as long as we clear it.
austin_model.policy.reset()

#Clearing an instance of SimReplication is a bit tricky, so
#   be careful of this nuance. The following reset() method
#   clears the replication ("zero-ing" any saved data
#   as well as the current time).
# However, the randomly sampled parameters remain the same!
#   These are untouched.
# The random number generator is also untouched after
#   reset(), so simulating rep will draw random numbers
#   from where the random number generator last left off
#   (before the reset).
austin_model.simulate_time_period(945)

# Note that we are using different random numbers, so the
#    outcome is different from the previous replication.
austin_model.vaccine_groups[0].S

array([[30669.,  2255.],
       [27387.,  1967.],
       [ 9258.,  1660.],
       [ 1745.,  1624.],
       [  477.,   383.]])

In [139]:
# Due to the nuances of the random number generation,
#   in many cases it is more straightforward and less
#   risky to simply create a new replication rather than
#   reset the replication. We recommend this particularly
#   when debugging and comparing numbers during code
#   testing (since being careful with random number
#   generation is necessary for perfectly replicable
#   results).

# We also discuss the SimModel instance attribute
#   fixed_kappa_end_date, a nonnegative integer.
#   This value corresponds to the last day
#   at which historical data are used for transmission reduction
#   (and cocooning) parameters. By default, this value is 0,
#   which means that no historical data is used for these
#   parameters, and the parameter values from t = 1 onwards
#   are dictated by the tiers in the MultiTierPolicy object
#   attached to the simulation.
# Note that the upper bound on fixed_kappa_end_date is the
#   number of days of historical data available.
# Also note that if a simulation replication is being run
#   at timepoints t > fixed_kappa_end_date, there must be a
#   MultiTierPolicy attached.
# fixed_kappa_end_date allows us to "play peek-a-boo"
#   with historical data -- it allows us to incorporate a smaller
#   subset of the historical data available rather than the
#   entire set of historical data available.
# Below is an example of running a simulation replication
#   with historical transmission reduction for the first 100 days
#   and then with simulated tier-dictated transmission reduction
#   afterwards.
austin_model.policy.reset()
austin_model.reset()

austin_model.fixed_kappa_end_date = 100
austin_model.simulate_time_period(500)

# A SimModel's method compute_feasibility() checks for ICU capacity
#   violations from its fixed_kappa_end_date to its next_t. In this way,
#   we only penalize ICU capacity violations during the time period that
#   a MultiTierPolicy attached to the SimReplication instance
#   is actually in effect.
austin_model.compute_feasibility()

True

In [140]:
# We can also look at the tier history of a policy after simulation
# These are the stages that the staged-alert policy was in over time
austin_model.policy.tier_history

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
